In [ ]:
#export
from local.imports import *
from local.core import *
from local.layers import *
from local.data.pipeline import *
from local.data.source import *
from local.data.core import *
from local.optimizer import *
from local.learner import *
from torch.utils.data import TensorDataset

In [ ]:
# default_exp utils.test

In [ ]:
#hide
torch.cuda.set_device(int(os.environ.get('DEFAULT_GPU') or 0))

# Synthetic Learner

> For quick testing of the training loop and Callbacks

In [ ]:
# export
def synth_data(a=2, b=3, bs=16, n_trn=10, n_val=2, cuda=False):
    x_trn = torch.randn(bs*n_trn, 1)
    y_trn = a*x_trn + b + 0.1*torch.randn(bs*n_trn, 1)
    x_val = torch.randn(bs*n_val, 1)
    y_val = a*x_val + b + 0.1*torch.randn(bs*n_val, 1)
    train_ds = TensorDataset(x_trn, y_trn)
    valid_ds = TensorDataset(x_val, y_val)
    tfms = [Cuda()] if cuda else None
    train_dl = TfmdDL(train_ds, bs=bs, shuffle=True, tfms=tfms)
    valid_dl = TfmdDL(valid_ds, bs=bs, tfms=tfms)
    return DataBunch(train_dl, valid_dl)

In [ ]:
# export
class RegModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.a,self.b = nn.Parameter(torch.randn(1)),nn.Parameter(torch.randn(1))
    def forward(self, x): return x * self.a + self.b

In [ ]:
# export
def synth_learner(n_trn=10, n_val=2, cuda=False, **kwargs):
    return Learner(RegModel(), synth_data(n_trn=n_trn,n_val=n_val, cuda=cuda), MSELossFlat(), 
                   opt_func=partial(SGD, mom=0.9), **kwargs)